# [IAPR][iapr]: Lab 2 ‒  Object description


**Group ID:** 32

**Author 1 (sciper):** Julien Daniel Berger (247179)  
**Author 2 (sciper):** Ghali Chraibi (262251)   
**Author 3 (sciper):** Yasser Haddad (272292)   

**Release date:** 26.03.2021  
**Due date:** 23.04.2021 


## Important notes

The lab assignments are designed to teach practical implementation of the topics presented during class well as preparation for the final project, which is a practical project which ties together the topics of the course. 

As such, in the lab assignments/final project, unless otherwise specified, you may, if you choose, use external functions from image processing/ML libraries like opencv and sklearn as long as there is sufficient explanation in the lab report. For example, you do not need to implement your own edge detector, etc.

**! Before handling back the notebook !** rerun the notebook from scratch `Kernel` > `Restart & Run All`


[iapr]: https://github.com/LTS5/iapr

# Imports

In [ ]:
import tarfile
import os

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from numpy import sign, log10

import cv2
import skimage.io
from skimage.exposure import histogram
from skimage import restoration
from scipy.signal import convolve2d as conv2

## 0. Extract relevant data
We first need to extract the `lab-02-data.tar.gz` archive.
To this end, we use the [tarfile] module from the Python standard library.

[tarfile]: https://docs.python.org/3.6/library/tarfile.html

In [ ]:
data_base_path = 'data'
data_folder = 'lab-02-data'
data_part1 = os.path.join(data_base_path, data_folder, 'part1')
data_part2 = os.path.join(data_base_path, data_folder, 'part2')

tar_path = os.path.join(data_base_path, data_folder + '.tar.gz')
with tarfile.open(tar_path, mode='r:gz') as tar:
    tar.extractall(path=data_base_path)

---
## Part 1
In the `lab-02-data/part1` folder, you will find 28x28 grey-scale pictures of handwritten "0" and "1".
These digits have been extracted from MNIST dataset (http://yann.lecun.com/exdb/mnist/).

Your goal is to extract, from each of those images, a 2-dimensional feature vector (i.e. 2 features) and to plot them all on a 2D graph.
If you have chosen good features, the vectors of the "0"'s should nicely cluster in one part of the plane and those of the "1"'s in another.

Please try:
1. Fourier Descriptors (15pts). 
    1. Implementation (10 pts).
    2. Showing invariance to rotation, translation and scaling (5 pts).
2. Additional method of your choice (5 pts)


**Note:** for the Fourier descriptors, the u_k signal has to be constructed by following the contour point after point. Some pre-processing (image binarization, possibly some Mathematical Morphology) might be useful.

### 1.1 Data visualization

In [ ]:
def load(path, digit='0'):
    digit_path = os.path.join(path, digit)
    digit_names = [nm for nm in os.listdir(digit_path) if '.png' in nm]  # make sure to only load .png
    digit_names.sort()  # sort file names
    ic = skimage.io.imread_collection([os.path.join(digit_path, nm) for nm in digit_names])
    digit_im = skimage.io.concatenate_images(ic)
    return digit_im, digit_names
                        
#  Load zeros and ones
zeros_im, zeros_names = load(data_part1, digit='0')
ones_im, ones_names = load(data_part1, digit='1')


# Plot images
fig, axes = plt.subplots(2, len(zeros_im), figsize=(12, 3))
for ax, im, nm in zip(axes[0], zeros_im, zeros_names):
    ax.imshow(im, cmap='gray')
    ax.axis('off')
    ax.set_title(nm)
for ax, im, nm in zip(axes[1], ones_im, ones_names):
    ax.imshow(im, cmap='gray')
    ax.axis('off')
    ax.set_title(nm)

### 1.2 Fourier descriptors (15 pts)

#### 1.2.A Implementation

In [ ]:
def fourier_descriptors(img):
    # Find the contours of the image
    contours, _ = cv2.findContours(
        img,
        cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_NONE)
    contour_array = contours[0][:, 0, :]
    
    # We convert the contour coordinates into a complex representation
    contour_complex = np.empty(contour_array.shape[:-1], dtype=complex)
    contour_complex.real = contour_array[:, 0]
    contour_complex.imag = contour_array[:, 1]
    
    # We compute the Discrete Fast Fourier Transform of the complex contour and get the descriptors
    fourier_descript = np.fft.fft(contour_complex)
    
    return fourier_descript

In [ ]:
def get_amplitude(fourier_descript):
    amplitudes = []
    for f in fourier_descript:
        # For each descriptor, get the amplitude of the fourier transform
        amplitudes.append(np.sqrt(f.real*f.real + f.imag*f.imag))
    
    return amplitudes

In [ ]:
def get_two_features(imgs):
    f1 = []
    f2 = []
    for img in imgs:
        fourier_descript = fourier_descriptors(img)
        amplitudes = get_amplitude(fourier_descript)
        # Take only features 1 and 2 (we exclude feature 0)
        f1.append(amplitudes[1])
        f2.append(amplitudes[2])
    
    return f1, f2

Without pre-processing

In [ ]:
f1_zeros, f2_zeros = get_two_features(zeros_im)
f1_ones, f2_ones = get_two_features(ones_im)

In [ ]:
# Plot descriptors
fig = plt.figure(1, figsize=(12, 5))

plt.scatter(f1_zeros, f2_zeros, label="Zero", color='red')    
plt.scatter(f1_ones, f2_ones, label="One", color='blue')    

plt.legend()
plt.show()

With pre-processing

In [ ]:
hist, hist_centers = histogram(ones_im[1])
plt.plot(hist)
plt.title("Histogram of the brain image")
plt.xlabel("intensity [grayscale]")
plt.ylabel("number of pixels")
plt.show()

In [ ]:
#  Load zeros and ones
zeros_im, zeros_names = load(data_part1, digit='0')
ones_im, ones_names = load(data_part1, digit='1')

#img = zeros_im[0]
#img = ones_im[0]
#im = cv2.erode(img, kernel, iterations=1)
#im = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
#im = cv2.dilate(img,kernel,iterations = 1)

def thresholding(img, range1, range2):
    # Apply a threshold to keep only the pixels withing the indicated range
    thresh = cv2.inRange(img, range1, range2)
    
    return thresh
    

def restore(img):
    psf = np.ones((5, 5)) / 25
    t = conv2(img, psf, 'same')
    t += 0.1 * t.std() * np.random.standard_normal(t.shape)
    deconvolved, _ = restoration.unsupervised_wiener(t, psf)
    
    return deconvolved


def transform_opening(img, kernel_size=(3,3)):
    kernel = np.ones(kernel_size, np.uint8)
    t = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    
    return t

def transform_closing(img, kernel_size=(3,3)):
    kernel = np.ones(kernel_size, np.uint8)
    closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    
    return closing

def transform_high_pass(img):
    kernel = np.array([[0.0, -1.0, 0.0], 
                       [-1.0, 5.0, -1.0],
                       [0.0, -1.0, 0.0]])

    #kernel = kernel/(np.sum(kernel) if np.sum(kernel)!=0 else 1)
    t = cv2.filter2D(img, -1, kernel)
    
    return t

preprocessed_zeros = [transform_closing(thresholding(img, 127, 255), kernel_size=(2,2)) for img in zeros_im]
preprocessed_ones = [transform_closing(thresholding(img, 127, 255), kernel_size=(2,2)) for img in ones_im]

# Plot images
fig, axes = plt.subplots(4, len(zeros_im), figsize=(20, 4))
for ax, im in zip(axes[0], zeros_im):
    ax.imshow(im, cmap='gray')
    ax.axis('off')
    
for ax, im, nm in zip(axes[1], preprocessed_zeros, zeros_names):
    ax.imshow(im, cmap='gray')
    ax.axis('off')

for ax, im in zip(axes[2], ones_im):
    ax.imshow(im, cmap='gray')
    ax.axis('off')
    
for ax, im, nm in zip(axes[3], preprocessed_ones, ones_names):
    ax.imshow(im, cmap='gray')
    ax.axis('off')

In [ ]:
f1_zeros, f2_zeros = get_two_features(preprocessed_zeros)
f1_ones, f2_ones = get_two_features(preprocessed_ones)

In [ ]:
# Plot descriptors
fig = plt.figure(1, figsize=(12, 5))

plt.scatter(f1_zeros, f2_zeros, label="Zero", color='red')    
plt.scatter(f1_ones, f2_ones, label="One", color='blue')    

plt.legend()
plt.show()

### 1.3 Additional method (5 pts)

### Hu moments

Moments of an image allow to capture information about the shape of an object in a binary image. However, they are not invariant in *translation*, *scaling* and *rotation*. **Normalized central moments** are translation and scaling invariant, but not rotation invariant. 

This is where **Hu moments** come into play. They are based on a combination of normalized central moments and are translation, scaling and rotation invariant. They consist of 7 moments, the first 6 of which are also *reflection invariant*.

**Hu moments** are within a large range, and to be able to compare them and efficiently use them together, we use the following Log transform : $H_i = -\text{sign}(h_i) * \log_{10} |h_i|$.

In [ ]:
def get_hu_moments(img):
    """ 
    Computes and returns the transformed Hu moments of an image.
    
    The function first finds Hu moments and then applies a Log transform. 
    
    Parameters
    ----------
    img : np.ndarray (MxM)
        A 2D image

    Returns
    -------
    np.ndarray (7, 1)
        The transformed Hu moments of the image
    """
    # Calculate Moments
    moments = cv2.moments(img)
    # Calculate Hu Moments
    huMoments = cv2.HuMoments(moments)
    # Apply log transform for each moment
    for i in range(0, 7):
        huMoments[i] = -1 * sign(huMoments[i]) * log10(abs(huMoments[i]))
        
    return huMoments

def get_features_hu_moments(imgs, feature1_idx, feature2_idx):
    """ 
    Computes and returns 2 Hu moments for each image 
    
    Parameters
    ----------
    imgs : list(np.ndarray) (Nx(MxM))
        A list of 2D images
    
    feature1_idx : int
        Index of the

    Returns
    -------
    (list(float), list(float))
        The lists containing the requested Hu moments for each image
    """
    f1 = []
    f2 = []
    
    for img in imgs:
        huMoments = get_hu_moments(img)
        f1.append(huMoments[feature1_idx])
        f2.append(huMoments[feature2_idx])
        
    return f1, f2

To know which 2 moments to choose, we plot the combination of all moments.

In [ ]:
# Plot all combination of moments
fig, axes = plt.subplots(6, 6, figsize=(24, 24))

for i in range(7):
    for j in range(i+1, 7):
    
        f1_zeros, f2_zeros = get_features_hu_moments(zeros_im, i, j)
        f1_ones, f2_ones = get_features_hu_moments(ones_im, i, j)

        axes[i, j-1].scatter(f1_zeros, f2_zeros, label="Zero", color='red')    
        axes[i, j-1].scatter(f1_ones, f2_ones, label="One", color='blue') 
        axes[i, j-1].set_xlabel("Moment " + str(i+1), fontsize=14)
        axes[i, j-1].set_ylabel("Moment " +str(j+1), fontsize=14)

plt.tick_params(labelcolor="none", bottom=False, left=False)
#plt.subplots_adjust(bottom = 0.01, left=0.001)
plt.tight_layout()

plt.suptitle("Plot of all combinations of features", y=1.03, fontsize=20)
plt.show()

We notice that the 2d vector of moment 2 and 3 gives the most linearly separable populations in the plot.

In [ ]:
f1_zeros, f2_zeros = get_features_hu_moments(zeros_im, 1, 2)
f1_ones, f2_ones = get_features_hu_moments(ones_im, 1, 2)

fig = plt.figure(1, figsize=(12, 5))

plt.scatter(f1_zeros, f2_zeros, label="Zero", color='red')    
plt.scatter(f1_ones, f2_ones, label="One", color='blue')   

plt.xlabel("2nd Moment")
plt.ylabel("3rd Moment")

plt.title("Plot of the second and third Hu moments")
plt.legend()
plt.show()

---
## Part 2
The `lab-02-data/part2` folder contains grey-scale pictures of handwritten "2" and "3".
Extract the same feature (typically 2 Fourier descriptors) as in part 1 also on these images and plot them on the same graph as the features of the "0" and "1".
Is it possible to discriminate all these 4 digits with a 2-dimensional feature vector?

### 2.1 Data visualization

In [ ]:
#  Load twos and threes
twos_im, twos_names = load(data_part2, digit='2')
threes_im, threes_names = load(data_part2, digit='3')

# Plot images
fig, axes = plt.subplots(2, len(twos_im), figsize=(12, 3))
for ax, im, nm in zip(axes[0], twos_im, twos_names):
    ax.imshow(im, cmap='gray')
    ax.axis('off')
    ax.set_title(nm)
for ax, im, nm in zip(axes[1], threes_im, threes_names):
    ax.imshow(im, cmap='gray')
    ax.axis('off')
    ax.set_title(nm)

### 2.2 Fourier descriptors - 4 digits (10 pts)

In [ ]:
# Add your implementation and discussion